In [5]:
import numpy as np 
import cvxpy as cvx
import pandas as pd

In [6]:
qin = pd.read_parquet("data/forecast_df.par")["y"][:200].values

In [119]:
def LMI_book(A, B, Bw, C, D, alpha):
    # Get dimensions
    n, _ = A.shape  # System dimension
    _, m = B.shape  # Input dimension
    c_dim, _ = C.shape  # Output dimension
    _, d = Bw.shape  # Disturbance dimension
    
    # Define variables
    Q = cvx.Variable((n, n), symmetric=True)
    Y = cvx.Variable((m, n))
    F_d = cvx.Variable()
    BFw = cvx.vstack([np.zeros((1, d)), F_d * np.ones((1, d))])
    cvx_Bw = Bw + BFw
    sigma = cvx.Variable(nonneg=True)

    # First LMI block
    LMI_1 = cvx.bmat([
        [-Q + alpha*Q, np.zeros((n, d)), Q@A.T + Y.T@B.T],
        [np.zeros((d, n)), -alpha*np.eye(d), cvx_Bw.T],
        [A@Q + B@Y, cvx_Bw, -Q]
    ])

    # Second LMI block
    LMI_2 = cvx.bmat([
        [-Q, Q@C.T + Y.T@D.T],
        [C@Q + D@Y, -sigma*np.eye(c_dim)]
    ])

    # Define and solve the problem
    constraints = [
        Q >> 0,
        LMI_1 << 0,
        LMI_2 << 0
    ]
    
    problem = cvx.Problem(cvx.Minimize(sigma), constraints=constraints)
    problem.solve(solver=cvx.MOSEK)
    
    return Q.value, Y.value, sigma.value, BFw.value

In [120]:
def grad_est(alpha, batch_num, A, B, C, D, Bw, var=0.3, eps=1e-5):
    delta_f = []
    Q, Y, sigma, F_dp = LMI_book(A, B, Bw, C, D, alpha)
    for b in range(batch_num):
        mu_k = var*2*(np.random.random_sample() - 0.5)
        u_k = np.sign(mu_k)
        alpha_k = max(min(alpha+mu_k, 1), eps)
        Q_k, Y_k, sigma_k, F_dp_k = LMI_book(A, B, Bw, C, D, alpha_k)
        delta_f.append((sigma_k - sigma)/mu_k*u_k)
    return sum(delta_f)/batch_num

In [121]:
import numpy as np

ts = 1
a = 20

A = np.array([[1.0, 1/a],
             [0.0, 0.0]])

B = np.array([[-1/a],
             [0.0]])

B_w = np.array([[0, 1],
               [1, 0]])

C = np.array([[1.0, 0.0]])

In [127]:
a = 20
alpha = 0.01
Q, Y, sigma, F_dp = LMI_book(A, B, B_w, C, D, alpha)

In [131]:
K

array([[20.,  1.]])

In [129]:
K = Y @ np.linalg.inv(Q)
u = K @ x

In [130]:
(A+B*F_dp) + B_w*np.random.normal(0, 0.2)

array([[ 1.        ,  0.03953277],
       [-0.01046723,  0.        ]])